<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<img src = "https://github.com/Jangrae/img/blob/master/medical2.png?raw=true" width=800 align="left"/>

# 실습 내용

- 다양한 알고리즘으로 성능이 최적화된 모델을 만듭니다.
- 성능이 좋을 것으로 예상되는 모델을 선정해 새로운 데이터에 대해 예측을 수행합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

- 학습용 데이터를 불러옵니다.
- 평가용 데이터는 이후에 불러오게 됩니다.

In [ ]:
# 학습용 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/insurance_train.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

**데이터 설명**

- age: 나이
- sex: 성별(female, male)
- bmi: 체질량지수(체중을 키의 제곱으로 나눈 값, 적정수준:18.5 - 24.9)
- children: 자녀 수
- smoker: 흡연 여부
- region: 거주지역(northeast, southeast, southwest, northwest)
- **charges: 건강보험에서 지불한 의료비 - Target**

In [ ]:
# 데이터 살펴보기
data.head()

In [ ]:
# 기술통계 확인
data.describe()

In [ ]:
# 상관관계 
data.corr(numeric_only=True).style.background_gradient()

In [ ]:
# Target 분포 확인
plt.figure(figsize=(8, 4))
plt.subplot(2, 1, 1)
plt.hist(data['charges'], bins=30, alpha=0.7, ec='k')
plt.subplot(2, 1, 2)
plt.boxplot(x=data['charges'], vert=False)
plt.show()

In [ ]:
# 흡연자 비율
plt.figure(figsize=(8, 3))
sns.countplot(x=data['smoker'])
plt.show()

In [ ]:
# 남녀 비율
plt.figure(figsize=(8, 3))
sns.countplot(x=data['sex'])
plt.show()

In [ ]:
# 흡연 여부에 따른 의료비
plt.figure(figsize=(8, 3))
sns.histplot(x=data['charges'], hue=data['smoker'], bins=40)
plt.show()

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [ ]:
# target 확인
target = 'charges'

# 데이터 분리
x = data.drop(target, axis=1)
y = data.loc[:, target]

**2) 가변수화**

In [ ]:
# 가변수화
dumm_cols = ['sex', 'smoker', 'region']
x = pd.get_dummies(x, columns=dumm_cols, drop_first=True, dtype=int)

# 확인
x.head()

**3) 학습용, 평가용 데이터 분리**

In [ ]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 데이터 분리
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=1)

# 4.모델링

- 하이퍼파라미터 최적화 과정을 통해 최선의 성능을 갖는 모델을 만들고 성능을 검증합니다.
- 각 모델을 joblib.dump() 함수를 사용해 저장합니다.

In [ ]:
# xgboost 설치
# !pip install xgboost

In [ ]:
# lightgbm 설치
# !pip install lightgbm

- 이후 사용할 함수를 모두 불러옵니다.

In [ ]:
# 불러오기
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score

**1) Linear Regression**

- Linear Regression 알고리즘으로 모델링하고 성능을 검증합니다.
- 모델을 파일로 저장힙니다.

In [ ]:
# 선언하기
model = LinearRegression()

# 학습하기
model.fit(x_train, y_train)

# 예측하기
y_val_pred = model.predict(x_val)

# 평가하기
print('MAE:', mean_absolute_error(y_test, y_val_pred))
print('R2:', r2_score(y_test, y_val_pred))

In [ ]:
# 모델저장
joblib.dump(model, 'model_linear.pkl')

**2) Decision Tree**

- 하이퍼파라미터 최적화 과정을 통해 최선의 성능을 갖는 Decision Tree 모델을 만들고 성능을 검증하세요.
- 변수 중요도를 시각화해 확인하고, 모델을 파일로 저장힙니다.

In [ ]:
# 파라미터 선언
param = {'max_depth': range(1, 21)}

# 선언하기
model = GridSearchCV(DecisionTreeRegressor(),
                     param,
                     cv=5)

# 학습하기
model.fit(x_train, y_train)

# 결과확인
print('* 파라미터:', model.best_params_)
print('* 예측성능:', model.best_score_)

# 예측하기
y_val_pred = model.predict(x_val)

# 평가하기
print('* MAE:', mean_absolute_error(y_test, y_val_pred))
print('* R2:', r2_score(y_test, y_val_pred))

In [ ]:
# 변수 중요도 시각화
plt.figure(figsize=(3, 5))
plt.barh(list(x), model.best_estimator_.feature_importances_)
plt.show()

In [ ]:
# 모델저장
joblib.dump(model, 'model_tree.pkl')

**3) Random Forest**

- 하이퍼파라미터 최적화 과정을 통해 최선의 성능을 갖는 Random Forest 모델을 만들고 성능을 검증하세요.
- 변수 중요도를 시각화해 확인하고, 모델을 파일로 저장힙니다.

In [ ]:
# 파라미터 선언
param = {'max_depth': range(1, 21)}

# 선언하기
model = GridSearchCV(RandomForestRegressor(),
                     param,
                     cv=5)

# 학습하기
model.fit(x_train, y_train)

# 결과확인
print('* 파라미터:', model.best_params_)
print('* 예측성능:', model.best_score_)

# 예측하기
y_val_pred = model.predict(x_val)

# 평가하기
print('* MAE:', mean_absolute_error(y_test, y_val_pred))
print('* R2:', r2_score(y_test, y_val_pred))

In [ ]:
# 변수 중요도 시각화
plt.figure(figsize=(3, 5))
plt.barh(list(x), model.best_estimator_.feature_importances_)
plt.show()

In [ ]:
# 모델저장
joblib.dump(model, 'model_forest.pkl')

**4) LightGBM**

- 하이퍼파라미터 최적화 과정을 통해 최선의 성능을 갖는 LightGBM 모델을 만들고 성능을 검증하세요.
- 변수 중요도를 시각화해 확인하고, 모델을 파일로 저장힙니다.

In [ ]:
# 파라미터 선언
param = {'max_depth': range(1, 21)}

# 선언하기
model = GridSearchCV(LGBMRegressor(verbose=-1),
                     param,
                     cv=5)

# 학습하기
model.fit(x_train, y_train)

# 결과확인
print('* 파라미터:', model.best_params_)
print('* 예측성능:', model.best_score_)

# 예측하기
y_val_pred = model.predict(x_val)

# 평가하기
print('* MAE:', mean_absolute_error(y_test, y_val_pred))
print('* R2:', r2_score(y_test, y_val_pred))

In [ ]:
# 변수 중요도 시각화
plt.figure(figsize=(3, 5))
plt.barh(list(x), model.best_estimator_.feature_importances_)
plt.show()

In [ ]:
# 모델저장
joblib.dump(model, 'model_lgbm.pkl')

# 5.성능평가

- 가장 좋은 성능을 보였던 모델을 joblib.load() 함수를 사용해 불러옵니다.
- 불러온 모델로 새로운 데이터에 대해 예측을 수행합니다.

In [ ]:
# 모델 불러오기
model = joblib.load('model_forest.pkl')

In [ ]:
# 평가용 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/insurance_test.csv'
new_data = pd.read_csv(path)

# 확인
new_data.head()

- 원본 데이터프레임을 유지하기 위해 새로운 데이터프레임으로 복사합니다.
- 복사된 데이터프레임에 대해 가변수화를 수행합니다.

In [ ]:
# 데이터 복사
x_test = new_data.copy()

# 가변수화
dumm_cols = ['sex', 'smoker', 'region']
x_test = pd.get_dummies(x_test, columns=dumm_cols, drop_first=True, dtype=int)

# 확인
x_test.head()

- 가변수화된 결과를 대상으로 예측을 수행합니다.
- 참고: 예측을 위해서는 모델이 학습했던 데이터(x_train)과 데이터 구조가 같아야 합니다.

In [ ]:
# 예측하기
y_pred = model.predict(x_test)

# 확인
y_pred[:10]

- 예측 결과를 'charges_pred' 이름의 열로 원본 데이터프레임에 추가합니다.

In [ ]:
# 결과 합기기
new_data['charges_pred'] = y_pred

# 확인
new_data.head()